# API Face Samples

## Objective
This script adds a face to a face collection (large face list and large person group) using the Azure Face API. If more than one face is detected in the image, the largest face will be added.

In [ ]:
import os
from PIL import Image
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient, FaceAdministrationClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel, FaceAttributeTypeRecognition04, QualityForRecognition

def detect_faces(subscription_key, endpoint, image_path):
    with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=add-face-to-collection"}) as face_client:
        with open(image_path, 'rb') as image_data:
            detected_faces = face_client.detect(
                    image_content=image_data.read(),
                    detection_model=FaceDetectionModel.DETECTION03,
                    recognition_model=FaceRecognitionModel.RECOGNITION04,
                    return_face_id=True,
                    return_face_attributes=[FaceAttributeTypeRecognition04.QUALITY_FOR_RECOGNITION]
                )
        return detected_faces
    
def enlarge_bounding_box(face_rectangle, image_width, image_height, enlargement_factor=1.2):
    left = max(0, face_rectangle.left - (face_rectangle.width * (enlargement_factor - 1) / 2))
    top = max(0, face_rectangle.top - (face_rectangle.height * (enlargement_factor - 1) / 2))
    width = min(image_width - left, face_rectangle.width * enlargement_factor)
    height = min(image_height - top, face_rectangle.height * enlargement_factor)
    return {'left': int(left), 'top': int(top), 'width': int(width), 'height': int(height)}

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height
   
def add_face(subscription_key, endpoint, image_path, target_type, target_id, person_id=None, quality_filter=False):   
    target_face = None    
    faces = detect_faces(subscription_key, endpoint, image_path)
    if len(faces) == 0:
        return "No faces detected in the image."
    else:
        if quality_filter and faces[0].face_attributes.quality_for_recognition == QualityForRecognition.LOW:
            return "Face quality is too low. Please use a different image."
    
    if len(faces) > 1:
        image_width, image_height = get_image_dimensions(image_path)
        # If multiple faces are detected, use the first face (largest face) for adding to the target
        print(f"Multiple faces detected. Using the first face (largest face) for adding to the target.")
        face_rectangle = enlarge_bounding_box(faces[0].face_rectangle, image_width, image_height)
        target_face = [face_rectangle['left'],face_rectangle['top'],face_rectangle['width'], face_rectangle['height']]
    else:
        print(f"One face detected. Adding to the target.")

    if target_type == "large_face_list" or target_type == "large_person_group":
        with FaceAdministrationClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=add-face-to-collection"}) as face_admin_client:
            if target_type == "large_face_list":
                with open(image_path, 'rb') as image:
                    result = face_admin_client.large_face_list.add_face(
                        target_id,
                        image,
                        target_face=target_face,
                        detection_model=FaceDetectionModel.DETECTION03,
                    )
                poller_face_list = face_admin_client.large_face_list.begin_train(
                    large_face_list_id=target_id,
                    polling_interval=5,
                )
                poller_face_list.wait()
            elif target_type == "large_person_group":
                if not person_id:
                    return "Person ID is required for adding to a large person group."
                with open(image_path, 'rb') as image:
                    result = face_admin_client.large_person_group.add_face(
                        target_id,
                        person_id,
                        image,
                        target_face=target_face,
                        detection_model=FaceDetectionModel.DETECTION03,
                    )
                poller_person_group = face_admin_client.large_person_group.begin_train(
                    large_person_group_id=target_id,
                    polling_interval=5,
                )
                poller_person_group.wait()
            return result
    else:
        return "Invalid target type specified."

# Example usage
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
image_path = 'path_to_image.jpg'
target_type = 'large_face_list'  # or 'large_person_group'
target_id = 'your_large_face_list_id'  # or 'your_large_person_group_id'
person_id = 'None'  # Required if target_type is 'large_person_group'

result = add_face(FACE_KEY, FACE_ENDPOINT, image_path, target_type, target_id, person_id, quality_filter=True)
print(result)